In [1]:
import pandas as pd
import glob

### Import and concatenate all NO2 csv files into one

In [2]:
# Path to the directory containing CSV files 
csv_files_path = r"C:\Users\stlva\Documents\Python_Scripts\Data_analytics_bootcamp\air-quality-and-weather-analysis\Data\NO2\*.csv" # Adjust this path as necessary 

# Use glob to get all CSV file paths 
csv_files = glob.glob(csv_files_path) 

# Use list comprehension to read all CSV files and concatenate them into a single DataFrame 
NO2_df = pd.concat([pd.read_csv(file,low_memory=False) for file in csv_files], ignore_index=True) 

# Now NO2_df contains all the data from the 20 CSV files 


### Data cleaning and preprocessing

In [3]:
# Drop unwanted columns
NO2_df.drop(['Parameter Code', 
            'Sample Duration', 
            'Pollutant Standard', 
            'Event Type', 
            'Observation Count', 
            'Observation Percent',
            'Method Code', 
            'Method Name',  
            'Local Site Name', 
            'Address', 
            'Date of Last Change'],axis=1,inplace=True)

In [4]:
# Drop Canada
NO2_df.drop(NO2_df[(NO2_df['State Code']=='CC')].index,inplace=True)

# Change State Code dtype to int64
NO2_df['State Code'] = NO2_df['State Code'].astype('int64')

# Drop Mexico, Virgin Islands and Puerto Rico
NO2_df.drop(NO2_df[(NO2_df['State Code']==80)|(NO2_df['State Code']==78)|(NO2_df['State Code']==72)].index,inplace=True)

In [5]:
# Aggregating so we have 1 measurement per station per day per parameter
NO2_df = NO2_df.groupby(['State Code', 
                       'State Name', 
                       'County Code', 
                       'County Name', 
                       'City Name', 
                       'Site Num', 
                       'Datum', 
                       'Latitude', 
                       'Longitude', 
                       'Date Local', 
                       'Parameter Name', 
                       'Units of Measure']).agg({'Arithmetic Mean':'mean', '1st Max Value':'max','1st Max Hour':'max','CBSA Name':'first','AQI':'mean'}).reset_index()

In [6]:
# Create unique identifier for each measurement
NO2_df['measurement_id'] = NO2_df['State Code'].apply(str) + '_' + NO2_df['County Code'].apply(str) + '_' + NO2_df['Site Num'].apply(str) + '_' + NO2_df['Date Local'].str.replace('-','')

In [7]:
# checking for the parameters included in the dataframe
NO2_df['Parameter Name'].unique()

array(['Nitrogen dioxide (NO2)'], dtype=object)

In [8]:
# Renaming column to match the value measured and droping unnecessary columns
NO2_df.rename(columns={'Arithmetic Mean':'NO2(ppm)','1st Max Value':'NO2(ppm) Max Value', '1st Max Hour':'NO2(ppm) Max Hour'},inplace=True)
NO2_df.drop(['Parameter Name', 'Units of Measure'],axis=1,inplace=True)

In [9]:
# Convert date column to datetime
NO2_df['Date Local'] = pd.to_datetime(NO2_df['Date Local'])

In [13]:
NO2_df.head()

,State Code,State Name,County Code,County Name,City Name,Site Num,Datum,Latitude,Longitude,Date Local,NO2(ppm),NO2(ppm) Max Value,NO2(ppm) Max Hour,CBSA Name,AQI,measurement_id
0,1,Alabama,71,Jackson,Not in a city,20,WGS84,34.877025,-85.720805,2004-02-01,3.521739,9.0,7,"Scottsboro, AL",8.0,1_71_20_20040201
1,1,Alabama,71,Jackson,Not in a city,20,WGS84,34.877025,-85.720805,2004-02-02,7.739130,17.0,22,"Scottsboro, AL",16.0,1_71_20_20040202
2,1,Alabama,71,Jackson,Not in a city,20,WGS84,34.877025,-85.720805,2004-02-03,4.086957,12.0,0,"Scottsboro, AL",11.0,1_71_20_20040203
3,1,Alabama,71,Jackson,Not in a city,20,WGS84,34.877025,-85.720805,2004-02-04,6.826087,13.0,18,"Scottsboro, AL",12.0,1_71_20_20040204
4,1,Alabama,71,Jackson,Not in a city,20,WGS84,34.877025,-85.720805,2004-02-05,5.608696,9.0,14,"Scottsboro, AL",8.0,1_71_20_20040205


In [14]:
#Saving final df to csv
#NO2_df.to_csv(r"C:\Users\stlva\Documents\Python_Scripts\Data_analytics_bootcamp\air-quality-and-weather-analysis\Cleaning\all_NO2_clean.csv",index=False)
NO2_df.to_csv(r"C:\Users\stlva\Google Drive\Big blue data academy\main_branch\concatenated data\all_NO2_clean.csv",index=False)